# Dynamic Contrast Enhanced Image Reconstruction with BART

In [ ]:
export PATH=${TOOLBOX_PATH}:${PATH}
export DEBUG_LEVEL=1

In this demo we will investigate prospectively under-sampled dynamic contrast enhanced (DCE) MRI data provided by [Tao
Zhang](http://web.stanford.edu/~tzhang08/). 

Unzip the data and it will be placed under `data/`.

In [ ]:
unzip data.zip

The data consist of a single slice from under-sampled multi-channel, Cartesian k-space binned into temporal phases, as well as
ESPIRiT sensitivity maps and sampling weights derived
from navigator data. 

We will first look at the datasets. BART provides a simple multi-dimensional viewer `bartview.py` in python. We also have a more sophisticated viewer `view` in development available at:
    https://github.com/mrirecon/view

In [ ]:
view data/ksp
bartview.py data/maps
view data/weights

We can see the dimensions using the `show` command:

In [ ]:
bart show -m data/ksp

The MRI-specific BART commands assume that the first three dimensions represent space/spatial k-space, the next two dimensions
represent coils and ESPIRiT maps, and the 10th dimension (zero-indexing) represents temporal phases.
We see the spatial dimensions are `1 X 68 x 180`, with 20 coils and 18 temporal phases.

To start, let's compute the sampling pattern and compare it to the k-space weighting mask.
The weighting mask assigns each phase encode a weight between zero and one, depending on the amount of
motion-corruption.

In [ ]:
bart pattern data/ksp data/pat # generate a sampling pattern from k-space
bart join 1 data/pat data/weights data/pat_weights
bart show -m data/pat_weights

We can sum along the temporal dimension using the `fmac` (fused multiply accumulate) tool:

In [ ]:
bart fmac -h

The bitmask is used to specify the dimensions we want to sum over. We can quickly compute a bitmask using the `bitmask`
tool:

In [ ]:
bart bitmask 10 # bitmask for the temporal dimensions
bart bitmask 0 1 2 # bitmask for 3D spatial dimensions

To sum over the temporal dimension, we create a BART file containing a single entry with value one, and sum over the
10th dimension:

In [ ]:
bart ones 1 1 o
bart fmac -s $(bart bitmask 10) data/pat_weights o data/pat_weights_sum
view data/pat_weights_sum & # visualize the result

We will use the `pics` tool to perform ESPIRiT-based parallel imaging and compressed sensing. The basic usage is

    bart pics [optimization options] [regularization options] kspace maps recon
    
See the full list with `bart pics -h` and `bart pics -Rh`.
There are several built-in regularization terms and transforms. We can see the options by invoking the help:

In [ ]:
bart pics -h

In [ ]:
bart pics -Rh

## L1-wavelets in space and total variation in time.

Initialize the viewer with zero-filled reconstruction

In [ ]:
bart pics -i 1 data/ksp data/maps recon_wavtv
view recon_wavtv&

In [ ]:
bart pics   -i 100 \
            -p data/weights \
            -R W:$(bart bitmask 0 1 2):0:0.001 \
            -R T:$(bart bitmask 10):0:.04 \
            data/ksp data/maps recon_wavtv


## Locally low rank (LLR).

Initialize the viewer with zero-filled reconstruction

In [ ]:
bart pics -i 1 data/ksp data/maps recon_llr
view recon_llr&

In [ ]:
bart pics   -i 100 \
            -R L:$(bart bitmask 0 1 2):$(bart bitmask 0 1 2):0.01 \
            data/ksp data/maps recon_llr

Combine the reconstructions and compare

In [ ]:
bart join 1 recon_wavtv recon_llr recon_compare # combine the reconstructions
view recon_compare &